In [35]:
import pandas as pd
import glob
from datetime import date, timedelta
import numpy as np
from datetime import datetime
from datetime import time
from pathlib import Path
from collections import OrderedDict
import polars as pl
import fastexcel
import xlsxwriter
import os
import networkx as nx

In [36]:
# def process_schedule(sheet_name):
#     df = pd.read_excel(r'C:/Users/ADMIN/OneDrive/Desktop/book1.xlsx', sheet_name=sheet_name)
#     df = df[[col for col in df.columns if not str(col).lower().startswith("shift")]]
#     id_vars = ['IEX ID', 'Name','TL'] #, 'Status', 'Sec Status'
#     df_melted = pd.melt(df, id_vars=id_vars, var_name='Date', value_name='Shift')
#     df_melted = df_melted[df_melted['Shift'].notna()]
#     df_melted['LOB'] = sheet_name

#     return df_melted

# schedule_lodging = process_schedule("Lodging")
# schedule_nonlodging = process_schedule("Non_Lodging")
# combined_schedule = pd.concat([schedule_lodging, schedule_nonlodging], ignore_index=True)

# combined_schedule['Date'] = pd.to_datetime(combined_schedule['Date'], errors='coerce')
# combined_schedule['Week'] = combined_schedule['Date'] - pd.to_timedelta(combined_schedule['Date'].dt.weekday, unit='d')

# output_folder = "C:/Users/ADMIN/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/INPUT_SCHEDULE/SCHEDULE_VN"
# os.makedirs(output_folder, exist_ok=True)

# for week, group in combined_schedule.groupby('Week'):
#     week_str = week.strftime('%Y_%m_%d')
#     file_path = os.path.join(output_folder, f"Schedule_{week_str}.xlsx")
#     sheet_name = f"Schedule_{week_str}"

#     with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
#         pivot_schedule = group.pivot_table(
#             index=['IEX ID', 'Name', 'TL', 'LOB'], #'Status', 'Sec Status', 
#             columns='Date',
#             values='Shift',
#             aggfunc='first'
#         )
#         pivot_schedule.columns = pivot_schedule.columns.astype(str)

#         pivot_schedule.to_excel(writer, sheet_name=sheet_name, index=True)

In [37]:
def input_data(folder_path):
    file_paths = glob.glob(f"{folder_path}/*.xlsx") + glob.glob(f"{folder_path}/*.csv")
    df_list = []
    for file in file_paths:
        source_name = os.path.splitext(os.path.basename(file))[0]
        
        if file.endswith('.xlsx'):
            df = pl.read_excel(file, has_header=False)
            new_header = [str(item).strip() for item in df.row(0)]
            df = df.slice(1)
            df.columns = [str(col).strip() for col in new_header]
            
        elif file.endswith('.csv'):
            df = pl.read_csv(file)
            df = df.with_columns([pl.col(col).cast(pl.String) for col in df.columns])
        
        df = df.with_columns(pl.lit(source_name).alias("Week"))

        cols_to_choose = [col for col in ["Week", "IEX ID", "Name", "LOB", "TL"] if col in df.columns]
        cols_to_drop = [col for col in ['Status', 'Sec Status'] if col in df.columns]
        df = df.drop(cols_to_drop)
        
        df = df.melt(id_vars=cols_to_choose, variable_name="Date", value_name="Shift")
        
        df = df.with_columns(
            pl.coalesce([
                pl.col("Date").str.strptime(pl.Date, "%Y-%m-%d", strict=False),
                pl.col("Date").str.strptime(pl.Date, "%Y-%m-%d %H:%M:%S", strict=False)
            ]).alias("Date")
        )
        df_list.append(df)
    
    if df_list:
        merged_df = pl.concat(df_list, how='vertical')
    else:
        merged_df = pl.DataFrame()  # Tránh lỗi nếu không có file nào
    
    return merged_df

def process_schedule_shift(df, shift_column_name):
    mask = df[shift_column_name].str.contains("-", na=False)
    
    df["Start_Shift"] = df[shift_column_name].where(mask).str.split("-").str[0]
    df["End_Shift"] = df[shift_column_name].where(mask).str.split("-").str[1]
    df["Start_Shift"] = pd.to_datetime(df["Start_Shift"], format="%H%M", errors="coerce").dt.time
    df["End_Shift"] = pd.to_datetime(df["End_Shift"], format="%H%M", errors="coerce").dt.time
    def classify_day_type(shift_time):
        if isinstance(shift_time, time):
            return "Night_Shift" if shift_time >= time(18, 0) else "Day_Shift"
        return np.nan
    def classify_shift_type(shift_time):
        if shift_time is None:
            return None
        if isinstance(shift_time, time):
            if time(5, 0) <= shift_time < time(12, 0):
                return "Morning"
            elif time(12, 0) <= shift_time < time(18, 0):
                return "Afternoon"
            elif shift_time >= time(18, 0):
                return "Night"
            else:
                return "Day_Shift"
        return None
    df["Day_Type"] = df["Start_Shift"].apply(classify_day_type)
    df["Shift_Type"] = df["Start_Shift"].apply(classify_shift_type)
    return df

In [38]:
first_glob = "C:/Users/huuchinh.nguyen"
# first_glob = "C:/Users/ADMIN"

week = "Schedule_2025_06_23"

folder_paths = {
    "schedule_team":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/INPUT_SCHEDULE/SCHEDULE_TEAM',
    "vn_team":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Schedule/Schedule (RTA version)/2025/2025-06/Schedule_WB0623.xlsx',
    "schedule_vn":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/INPUT_SCHEDULE/SCHEDULE_VN',
    "analysis":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/INPUT_SCHEDULE/SCHEDULE_ANALYSIS',
    "schedule_step_1":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/INPUT_SCHEDULE/SCHEDULE_STEP_1'
}

In [39]:
def load_and_merge_hc_data(excel_path: str) -> pd.DataFrame:
    selected_columns = ["Site","IEX ID", "OracleID", "Employee Name", "Alias", "Gender", "Email Id", "LWD/Movement", "Mini Team"]

    def read_and_process_sheet(sheet_name: str) -> pd.DataFrame:
        df = pd.read_excel(excel_path, sheet_name=sheet_name, header=None)
        df.columns = df.iloc[0]
        df = df[1:]
        for col in selected_columns:
            if col not in df.columns:
                df[col] = pd.NA
        df = df[selected_columns]
        return df

    active_df = read_and_process_sheet("Active")
    inactive_df = read_and_process_sheet("Inactive")
    combined_df = pd.concat([active_df, inactive_df], ignore_index=True)
    combined_df['LWD/Movement'] = pd.to_datetime(combined_df['LWD/Movement'], errors='coerce')
    combined_df['IEX ID'] = pd.to_numeric(combined_df['IEX ID'], errors='coerce')
    return combined_df

# hc = load_and_merge_hc_data(folder_paths['vn_team'])

In [40]:
def get_most_frequent_shift(df):
    expected_cols = ["Week", "IEX ID", "Name", "TL", "LOB", "Shift"]
    existing_cols = [col for col in expected_cols if col in df.columns]

    df = df.select(existing_cols)

    shift_with_dash = df.filter(pl.col("Shift").str.contains("-"))
    
    if shift_with_dash.height > 0:
        df_filtered = shift_with_dash
    else:
        df_filtered = df.with_columns(pl.lit("No Shift").alias("Shift"))
    
    result_df = (
        df_filtered.group_by(["Week", "IEX ID", "Name", "TL", "LOB", "Shift"])
                   .agg(pl.count().alias("Count"))
                   .sort(by=["Week", "IEX ID", "Name", "TL", "LOB", "Count"], 
                         descending=[False, False, False, False, False, True])
                   .filter(pl.col("Count") == pl.col("Count").max().over(["Week", "IEX ID", "Name", "TL", "LOB"]))
                   .drop("Count")
    )

    return result_df

In [41]:
def load_schedule_of_schedule(folder_path):
    all_schedules = []

    schedule_df = input_data(folder_path)
    schedule_df = get_most_frequent_shift(schedule_df)

    if schedule_df.height > 0:
        schedule_df = schedule_df.to_pandas()
        schedule_df = process_schedule_shift(schedule_df, "Shift")
        schedule_df['IEX ID'] = pd.to_numeric(schedule_df['IEX ID'], errors='coerce')
        all_schedules.append(schedule_df)

    return pd.concat(all_schedules, ignore_index=True) if all_schedules else pd.DataFrame()

Schedule_Of_Schedule_Team = load_schedule_of_schedule(folder_paths["schedule_team"])
Schedule_Of_Schedule_Team['Source'] = "Schedule_Team"

Schedule_Of_Schedule_VN = load_schedule_of_schedule(folder_paths["schedule_vn"])
Schedule_Of_Schedule_VN['Source'] = "VN_Team"

Combine_Schedule = pd.concat([Schedule_Of_Schedule_Team,Schedule_Of_Schedule_VN])

for week_col, group in Combine_Schedule.groupby('Week'):
    
    file_name = os.path.join(folder_paths['analysis'], f"{week_col}.xlsx")
    group.to_excel(file_name, sheet_name=str(week_col), index=False)

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25628\2713187756.py:23: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df = df.melt(id_vars=cols_to_choose, variable_name="Date", value_name="Shift")
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25628\2759199084.py:16: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("Count"))
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25628\2713187756.py:23: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df = df.melt(id_vars=cols_to_choose, variable_name="Date", value_name="Shift")
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25628\2759199084.py:16: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("Count"))


In [42]:
schedule_step_1 = pd.read_excel(f"{folder_paths['schedule_team']}/{week}.xlsx")
schedule_step_1['Week'] = week
cols = ['Week'] + [col for col in schedule_step_1.columns if col != 'Week']
Schedule_VN = schedule_step_1[cols]

Schedule_Swap = pd.read_excel(folder_paths['vn_team'], sheet_name="Swap", header=None)
Schedule_Swap.columns = Schedule_Swap.iloc[0]
Schedule_Swap = Schedule_Swap.iloc[1:].reset_index(drop=True)
Schedule_Swap = Schedule_Swap[Schedule_Swap['Week'] == week]

columns_to_select = ['Week', 'IEX1', 'IEX2','Name1','Name2','Step', 
                     'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
Schedule_Swap = Schedule_Swap[columns_to_select]

week_start_date = datetime.strptime(week.replace("Schedule_", ""), "%Y_%m_%d")
day_mapping = {
    "Monday": (week_start_date + timedelta(days=0)).strftime("%Y-%m-%d"),
    "Tuesday": (week_start_date + timedelta(days=1)).strftime("%Y-%m-%d"),
    "Wednesday": (week_start_date + timedelta(days=2)).strftime("%Y-%m-%d"),
    "Thursday": (week_start_date + timedelta(days=3)).strftime("%Y-%m-%d"),
    "Friday": (week_start_date + timedelta(days=4)).strftime("%Y-%m-%d"),
    "Saturday": (week_start_date + timedelta(days=5)).strftime("%Y-%m-%d"),
    "Sunday": (week_start_date + timedelta(days=6)).strftime("%Y-%m-%d"),
}
day_abbr = {
    "Monday": "Mon", 
    "Tuesday": "Tue", 
    "Wednesday": "Wed",
    "Thursday": "Thu", 
    "Friday": "Fri", 
    "Saturday": "Sat", 
    "Sunday": "Sun"
}

for _, row in Schedule_Swap.iterrows():
    iex1, iex2 = row["IEX1"], row["IEX2"]
    if iex1 == iex2:
        for day, date_col in day_mapping.items():
            new_value = row[day]
            Schedule_VN.loc[Schedule_VN["IEX ID"] == iex1, date_col] = new_value

def swap_shift(schedule_df, iex1, iex2, day_col):
    idx1 = schedule_df.index[schedule_df["IEX ID"] == iex1].tolist()
    idx2 = schedule_df.index[schedule_df["IEX ID"] == iex2].tolist()
    if not idx1 or not idx2:
        print(f"Không tìm thấy IEX ID {iex1} hoặc {iex2} trong Schedule_VN")
        return schedule_df
    idx1 = idx1[0]
    idx2 = idx2[0]

    val1 = schedule_df.at[idx1, day_col]
    val2 = schedule_df.at[idx2, day_col]

    schedule_df.at[idx1, day_col] = val2
    schedule_df.at[idx2, day_col] = val1

    return schedule_df

G = nx.Graph()
pair_day_map = {}
pair_step_map = {}

for _, row in Schedule_Swap.iterrows():
    iex1, iex2 = row["IEX1"], row["IEX2"]
    if iex1 != iex2:
        pair = tuple(sorted([iex1, iex2]))
        G.add_edge(*pair)
        swap_days = [day for day in day_mapping.keys() if row[day] == "SWAP"]
        if pair not in pair_day_map:
            pair_day_map[pair] = set()
        pair_day_map[pair].update(swap_days)

        if pair not in pair_step_map:
            pair_step_map[pair] = set()
        pair_step_map[pair].add(str(row["Step"]))

for pair, swap_days in pair_day_map.items():
    iex1, iex2 = pair
    for day in swap_days:
        date_col = day_mapping[day]
        Schedule_VN = swap_shift(Schedule_VN, iex1, iex2, date_col)

iex_to_detailed_history = {iex: [] for iex in Schedule_VN["IEX ID"].unique()}
iex_to_step = {iex: set() for iex in Schedule_VN["IEX ID"].unique()}
full_week = {"Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"}

for component in nx.connected_components(G):
    subgraph = G.subgraph(component)

    group_steps = set()
    for u, v in subgraph.edges():
        pair = tuple(sorted([u,v]))
        group_steps.update(pair_step_map.get(pair, []))

    group_detailed_steps = []

    for u, v in subgraph.edges():
        pair = tuple(sorted([u,v]))
        days = sorted(pair_day_map.get(pair, []))
        if set(days) == full_week:
            day_str = ""
        else:
            day_str = ", ".join([day_abbr[day] for day in days])
            day_str = f" (Swap {day_str})"

        def get_name(iex):
            name = Schedule_Swap.loc[Schedule_Swap["IEX1"] == iex, "Name1"]
            if name.empty:
                name = Schedule_Swap.loc[Schedule_Swap["IEX2"] == iex, "Name2"]
            return name.values[0] if not name.empty else ""

        steps = sorted(map(int, pair_step_map.get(pair, [])))

        for step in steps:
            line = f"{u} - {get_name(u)} <-> {v} - {get_name(v)}{day_str} (Step {step})"
            group_detailed_steps.append((step, line))

    group_detailed_steps.sort(key=lambda x: x[0])

    group_detailed_sorted_unique = []
    seen_lines = set()
    for _, line in group_detailed_steps:
        if line not in seen_lines:
            group_detailed_sorted_unique.append(line)
            seen_lines.add(line)

    for node in subgraph.nodes:
        iex_to_step[node].update(group_steps)
        iex_to_detailed_history[node] = group_detailed_sorted_unique.copy()

for _, row in Schedule_Swap.iterrows():
    iex1, iex2, step = row["IEX1"], row["IEX2"], str(row["Step"])
    if iex1 == iex2:
        name = row["Name1"]
        step_note = f"(Step {step})"
        line = f"{iex1} - {name} moved to new shift {step_note}"
        if line not in iex_to_detailed_history[iex1]:
            iex_to_detailed_history[iex1].append(line)
        iex_to_step[iex1].add(step)

iex_to_detailed_history = {k: "\n".join(v) if isinstance(v, list) else v for k, v in iex_to_detailed_history.items()}
iex_to_step = {k: ", ".join(sorted(v, key=int)) for k, v in iex_to_step.items()}

Schedule_VN["Detailed Swap History"] = Schedule_VN["IEX ID"].map(iex_to_detailed_history).fillna("")
Schedule_VN["Swap Steps"] = Schedule_VN["IEX ID"].map(iex_to_step).fillna("")

schedule_step_1[(schedule_step_1['Week'] == 'Schedule_2025_06_09') & (schedule_step_1['IEX ID'] == 3092416)]


C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25628\1443098593.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Schedule_VN["Detailed Swap History"] = Schedule_VN["IEX ID"].map(iex_to_detailed_history).fillna("")
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25628\1443098593.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Schedule_VN["Swap Steps"] = Schedule_VN["IEX ID"].map(iex_to_step).fillna("")


,IEX ID,Name,TL,Status,Sec Status,LOB,2025-06-23,2025-06-24,2025-06-25,2025-06-26,2025-06-27,2025-06-28,2025-06-29,Week


In [43]:
Schedule_VN[(schedule_step_1['Week'] == 'Schedule_2025_06_09') & (schedule_step_1['IEX ID'] == 3092416)]

,Week,IEX ID,Name,TL,Status,Sec Status,LOB,2025-06-23,2025-06-24,2025-06-25,2025-06-26,2025-06-27,2025-06-28,2025-06-29,Detailed Swap History,Swap Steps


In [44]:
Schedule_VN[(schedule_step_1['Week'] == 'Schedule_2025_06_09') & (schedule_step_1['IEX ID'] == 3092416)]

,Week,IEX ID,Name,TL,Status,Sec Status,LOB,2025-06-23,2025-06-24,2025-06-25,2025-06-26,2025-06-27,2025-06-28,2025-06-29,Detailed Swap History,Swap Steps


In [45]:
Schedule_VN[(schedule_step_1['Week'] == 'Schedule_2025_06_09') & (schedule_step_1['IEX ID'] == 3092416)]

,Week,IEX ID,Name,TL,Status,Sec Status,LOB,2025-06-23,2025-06-24,2025-06-25,2025-06-26,2025-06-27,2025-06-28,2025-06-29,Detailed Swap History,Swap Steps


In [46]:
Schedule_VN['Planned Note'] = ""
Schedule_Planned = pd.read_excel(folder_paths['vn_team'], sheet_name="Planned", header=None)
Schedule_Planned.columns = Schedule_Planned.iloc[0]
Schedule_Planned = Schedule_Planned[1:].reset_index(drop=True)
Schedule_Planned = Schedule_Planned[['Leave Date', 'Week','Type of Request', 'IEX ID', 'Agent Name','Planned']]
Schedule_Planned['Leave Date'] = pd.to_datetime(Schedule_Planned['Leave Date'] )
Schedule_Planned['Leave Date'] = Schedule_Planned['Leave Date'].astype(str)

for _, row in Schedule_Planned.iterrows():
    col_name = row['Leave Date']
    iex = row['IEX ID']
    req = row['Type of Request']
    name = row['Agent Name']

    if col_name in Schedule_VN.columns:
        mask = Schedule_VN['IEX ID'] == iex
        
        if not Schedule_VN.loc[mask, col_name].empty and Schedule_VN.loc[mask, col_name].iloc[0] != "OFF":
            Schedule_VN.loc[mask, col_name] = req
            Schedule_VN.loc[mask, 'Planned'] = "RTA"
        else:
            if Schedule_VN.loc[mask, 'Planned Note'].empty:
                existing_note = ""
            else:
                existing_note = Schedule_VN.loc[mask, 'Planned Note'].iloc[0]
            new_note = f"{iex} - {name} has {req} on OFF date {col_name}"
            
            if pd.isna(existing_note) or existing_note == "":
                Schedule_VN.loc[mask, 'Planned Note'] = new_note
            else:
                Schedule_VN.loc[mask, 'Planned Note'] = existing_note + " | " + new_note
    else:
        print(f"Cột '{col_name}' does not exist in Schedule_VN!!")

cols_to_check = Schedule_VN.columns[7:14]

Schedule_VN['Planned'] = Schedule_VN[cols_to_check].apply(lambda x: (x == 'AL').sum() + (x == 'CO').sum(), axis=1)

Schedule_VN['Week'] = week
columns_to_replace = Schedule_VN.columns[6:]
Schedule_VN[columns_to_replace] = Schedule_VN[columns_to_replace].replace({'PTO': 'AL','Paid Leave': 'CO'})
Schedule_VN['Planned'] = Schedule_VN[cols_to_check].apply(lambda x: x[x.str.contains('AL|CO', na=False)].count(), axis=1)

file_path = os.path.join(folder_paths['schedule_step_1'], week + ".xlsx")

output_path = f"{folder_paths['schedule_step_1']}/{week}.xlsx"
Schedule_VN.to_excel(output_path, index=False)

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25628\2174258964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Schedule_VN['Planned Note'] = ""


Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-19' does not exist in Schedule_VN!!
Cột '2025-05-20' does not exist in Schedule_VN!!
Cột '2025-05-20' does not exist in Schedule_VN!!
Cột '2025-05-20' does not exist in Schedule_VN!!
Cột '2025-05-20' does not exist in Schedule_VN!!
Cột '2025-05-20' does not exist in Schedule_VN!!
Cột '2025-05-20' does not exist in Schedule_VN!!
Cột '2025-05-20' does not exist in Schedule_VN!!
Cột '2025-05-20' does not exist in Schedule_VN!!
Cột '2025-05-21' does not exist in Schedule_VN!!
Cột '2025-05-21' doe

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25628\2174258964.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Schedule_VN.loc[mask, 'Planned'] = "RTA"
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25628\2174258964.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Schedule_VN['Planned'] = Schedule_VN[cols_to_check].apply(lambda x: (x == 'AL').sum() + (x == 'CO').sum(), axis=1)
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25628\2174258964.py:39: SettingWithCopyWarning: 
A val

In [47]:
# Schedule_Tool = pd.read_excel(folder_paths['vn_team'], sheet_name="Check_Schedule_VN", header=None)

# Schedule_Tool = Schedule_Tool[2:]
# Schedule_Tool.columns = Schedule_Tool.iloc[0]
# Schedule_Tool = Schedule_Tool[1:]

# cols_to_check = Schedule_Tool.iloc[:, 6:]

# def check_7_days(row):
#     count = 0
#     for value in row:
#         if "-" in str(value): 
#             if count == 7:
#                 return 1
#         else:
#             count = 0
#     return 0

# Schedule_Tool['7 Days'] = cols_to_check.apply(check_7_days, axis=1)


# def count_short_rest_periods(df, start_col_index=6):
#     short_rest_count = []

#     for idx, row in df.iterrows():
#         count = 0
#         for i in range(start_col_index, len(df.columns) - 1):
#             col_curr = df.columns[i]
#             col_next = df.columns[i + 1]
#             time_curr = str(row[col_curr])
#             if '-' not in time_curr:
#                 continue
#             end_hour = time_curr[-4:-2]
#             end_min = time_curr[-2:]
#             end_datetime = pd.to_datetime(f"{col_curr} {end_hour}:{end_min}", errors='coerce')
#             time_next = str(row[col_next])
#             if '-' not in time_next:
#                 continue
#             start_hour = time_next[:2]
#             start_min = time_next[2:4]
#             start_datetime = pd.to_datetime(f"{col_next} {start_hour}:{start_min}", errors='coerce')
#             if pd.notna(end_datetime) and pd.notna(start_datetime):
#                 diff_hours = (start_datetime - end_datetime).total_seconds() / 3600
#                 if 0 < diff_hours < 12:
#                     count += 1
#         short_rest_count.append(count)

#     df['12 Hours'] = short_rest_count
#     return df

# Schedule_Tool = count_short_rest_periods(Schedule_Tool, start_col_index=6)